INSTALL
- FAISS - 
- Sent2Vec

In [1]:
import numpy as np
import faiss   

In [2]:
import os
import time
name = "Buddhism-bert-en-vi-0.999-"+ str(time.strftime("%Y%m%d-%H%M"))

!mkdir $name
path = os.getcwd() + "/" + name
os.chdir(path)

!pwd


os.chdir(path)


# import os
# os.chdir("/home/lw/clc_fairseq/TED-20210121-1335")

/home/chinh/clc_data/Script/Buddhism-bert-en-vi-0.999-20210408-1735


In [3]:
!wget https://github.com/hoangtrungchinh/clc_data/raw/master/buddhism.tar.gz
!mkdir data_bin
!tar -xvf  'buddhism.tar.gz' -C 'data_bin'

--2021-04-08 17:35:14--  https://github.com/hoangtrungchinh/clc_data/raw/master/buddhism.tar.gz
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/buddhism.tar.gz [following]
--2021-04-08 17:35:15--  https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/buddhism.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 910043 (889K) [application/octet-stream]
Saving to: ‘buddhism.tar.gz’

buddhism.tar.gz     100%[===================>] 888,71K  4,71MB/s    in 0,2s    

2021-04-08 17:35:16 (4,71 MB/s) - ‘buddhism.tar.gz’ saved [910043/910043]

Buddhism.vi


In [4]:
# import file
import numpy as np
import glob

en_files = sorted(glob.glob("data_bin/*.en"))
vi_files = sorted(glob.glob("data_bin/*.vi"))
print(en_files)
print(vi_files)

lst_en = []
lst_vi = []

start = time.time()

for file in en_files:
  with open(file) as file_in:
    for line in file_in:
      lst_en.append(line)

for file in vi_files:
  with open(file) as file_in:
    for line in file_in:
      lst_vi.append(line)

print(len(lst_en))
print(len(lst_vi))

['data_bin/Buddhism.en']
['data_bin/Buddhism.vi']
12880
12880


In [5]:
# SPLIT TRAIN, TEST, VALID
from sklearn.model_selection import train_test_split
en_train, en_test_valid, vi_train, vi_test_valid =  train_test_split(lst_en, lst_vi, test_size=0.2, random_state=123)
en_valid, en_test, vi_valid, vi_test = train_test_split(en_test_valid, vi_test_valid, test_size=0.5, random_state=123)
print(len(en_train))
print(len(vi_train))
print(len(en_valid))
print(len(vi_valid))
print(len(en_test))
print(len(vi_test))
print(en_train[20])
print(vi_train[20])
print(en_valid[-1])
print(vi_valid[-1])
print(en_test[-1])
print(vi_test[-1])

10304
10304
1288
1288
1288
1288
Every day after lunch, I helped the two temple boys clean the kitchen, then I left for a two-mile walk to a temple in the next village, where the head monk would drill me in Pali lessons.

Mỗi ngày sau bữa ăn, tôi phụ hai cậu bé ở chùa dọn dẹp nhà bếp, rồi tôi phải đi bộ gần ba cây số để đến một ngôi chùa ở làng kế bên, nơi vị trụ trì ở đó sẽ dạy tiếng Pali cho tôi.

If we are earnest, we should understand this problem of experience.

Nếu chúng ta nghiêm túc, chúng ta nên hiểu rõ vấn đề của trải nghiệm này.

You may be very generous in giving a large donation to your society, to your friends, but awfully mean about giving a bigger tip—you know what I mean by ‘mean’.

Bạn có lẽ rất quảng đại khi trao một hiến dâng to tát cho xã hội của bạn, cho bạn bè của bạn, nhưng rất keo kiệt khi cho một món quà lớn hơn – bạn biết tôi có ý gì qua từ ngữ ‘keo kiệt’.



In [6]:
file_variable = [en_train,vi_train,en_valid,vi_valid,en_test,vi_test]
file_names = ["en_train","vi_train","en_valid","vi_valid","en_test","vi_test"]

for i in range(len(file_names)):
  with open(file_names[i], 'w') as f:
    for item in file_variable[i]:
      f.write(item)
!ls -al

total 3972
drwxrwxr-x  3 chinh chinh    4096 Thg 4   8 17:35 .
drwxrwxr-x 15 chinh chinh    4096 Thg 4   8 17:35 ..
-rw-rw-r--  1 chinh chinh  910043 Thg 4   8 17:35 buddhism.tar.gz
drwxrwxr-x  2 chinh chinh    4096 Thg 4   8 17:35 data_bin
-rw-rw-r--  1 chinh chinh  127163 Thg 4   8 17:35 en_test
-rw-rw-r--  1 chinh chinh 1023485 Thg 4   8 17:35 en_train
-rw-rw-r--  1 chinh chinh  125859 Thg 4   8 17:35 en_valid
-rw-rw-r--  1 chinh chinh  182237 Thg 4   8 17:35 vi_test
-rw-rw-r--  1 chinh chinh 1486401 Thg 4   8 17:35 vi_train
-rw-rw-r--  1 chinh chinh  184873 Thg 4   8 17:35 vi_valid


In [7]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('stsb-bert-large')
# # model = SentenceTransformer('LaBSE')
# #Sentences are encoded by calling model.encode()
# emb1 = model.encode("This is a red cat with a hat.")
# emb2 = model.encode("Have you seen my red cat?")

# cos_sim = util.pytorch_cos_sim(emb1, emb2)
# print("Cosine-Similarity:", cos_sim)

In [8]:
# print(cos_sim.numpy()[0][0])

In [9]:
start = time.time()
srcfile = "en_train"
tarfile = "vi_train"
# lst_srcfile = open(srcfile, "r").readlines()
# lst_tarfile = open(tarfile, "r").readlines()
lst_srcfile = en_train
lst_tarfile = vi_train

vec_lst_srcfile = model.encode(lst_srcfile)
print('=== Ending, Total time (second): ', time.time() - start)

=== Ending, Total time (second):  96.60413241386414


Test Cosin similarity

In [10]:
import faiss
# print(vec_lst_srcfile.shape[1])
start = time.time()
index = faiss.IndexFlatL2(vec_lst_srcfile.shape[1])
index.ntotal
faiss.normalize_L2(vec_lst_srcfile)
index.add(vec_lst_srcfile)
k = 6
distance, index = index.search(vec_lst_srcfile, k)

print('=== Ending, Total time (second): ', time.time() - start)
print(len(distance))
print(len(index))

print(index)
print(distance)

=== Ending, Total time (second):  0.4561176300048828
10304
10304
[[    0  4920  4236  9814  1904   499]
 [    1    97  6311  7595  5109   457]
 [    2  2310  6262  2419  8504  5315]
 ...
 [10301  8713  1677  9679   550  9463]
 [10302  4614   782 10070  3096  7680]
 [10303  9930  8386  5544  9039  8971]]
[[0.0000000e+00 6.6155112e-01 6.6387594e-01 6.8464661e-01 7.0950115e-01
  7.0991087e-01]
 [0.0000000e+00 7.2325695e-01 8.4077001e-01 8.5588276e-01 8.8852787e-01
  8.9587438e-01]
 [1.1920929e-07 6.1776876e-01 6.3838422e-01 6.4054251e-01 6.4247692e-01
  7.0253527e-01]
 ...
 [3.5762787e-07 4.7707260e-01 6.1734474e-01 6.3555276e-01 6.5062296e-01
  6.8078601e-01]
 [0.0000000e+00 5.0135469e-01 5.5491328e-01 5.5865014e-01 6.1068690e-01
  6.4976466e-01]
 [0.0000000e+00 5.8159804e-01 6.0192490e-01 6.1349797e-01 6.5869880e-01
  6.9025135e-01]]


In [11]:
i = 6
j = k-1
print(index[i][j])
print("==================================")
print(distance[i][j])
a = vec_lst_srcfile[i]
# print(a)
b = vec_lst_srcfile[index[i][j]]
# TEST
dist_squared = np.sum(np.square(a - b))
print(dist_squared)

9740
0.7185621
0.718562


In [12]:
start = time.time()
saperate = " || "

src_label = "S"
emb_label = "E"
thres_arr = [0.8, 0.85, 0.9, 0.95, 0.99]
for i in range(len(thres_arr)):
    threshold = thres_arr[i]
    out_file_write = open("en_train_EM_"+str(thres_arr[i]), 'w')
    out_file_write_score = open("en_train_EM_score_"+str(thres_arr[i]), 'w')
    out_file_write_factor = open("en_train_EM_factor_"+str(thres_arr[i]), 'w')
    num_gth_thres = 0

    for i in range(len(lst_srcfile)):
        score = distance[i][1]
        content = ""
        content_label_arr = []
        if score >= threshold:
            num_gth_thres += 1
            best_simi_index = index[i][1]
            content = lst_srcfile[i].strip() + saperate + lst_tarfile[best_simi_index].strip()

            content_label = [src_label] * len(lst_srcfile[i].split())
            content_label.append(emb_label)
            content_label = content_label + [emb_label] * len(lst_tarfile[best_simi_index].split())

            content_label_arr = " ".join(content_label)
        else:
            content = lst_srcfile[i].strip()
            content_label = [src_label] * len(lst_srcfile[i].split())
            content_label_arr = " ".join(content_label)
 
        out_file_write.writelines(content+ "\n")    
        out_file_write_factor.writelines(content_label_arr + "\n")
        out_file_write_score.writelines(str(score) + "\n")

    print('threshold =', threshold, "EM sentences =", num_gth_thres, "/", len(lst_srcfile), num_gth_thres/len(lst_srcfile), "%" )

    out_file_write.close()
    out_file_write_score.close()
    out_file_write_factor.close()

done = time.time()
elapsed = done - start
print('=== Ending, Total time (second): ', elapsed)

threshold = 0.8 EM sentences = 1576 / 10304 0.1529503105590062 %
threshold = 0.85 EM sentences = 1027 / 10304 0.09967003105590062 %
threshold = 0.9 EM sentences = 616 / 10304 0.059782608695652176 %
threshold = 0.95 EM sentences = 358 / 10304 0.034743788819875776 %
threshold = 0.99 EM sentences = 224 / 10304 0.021739130434782608 %
=== Ending, Total time (second):  0.6767988204956055
